In [ ]:
# connect to postgresql db usingenvironment variable read from vars.env 

import warnings
warnings.filterwarnings("ignore")

import os
import psycopg2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# read environment variables from vars.env
from dotenv import load_dotenv
load_dotenv("../../vars.env")

# connect to postgresql db on localhost, post 5432, using user and password from vars.env

import psycopg2
import os

# Define the database credentials
db_host = os.getenv("POSTGRES_HOST")
db_name = os.getenv("POSTGRES_DB")
db_user = os.getenv("POSTGRES_USER")
db_password = os.getenv("POSTGRES_PWD")

# Create the connection
connection = psycopg2.connect(
    host=db_host,
    database=db_name,
    user=db_user,
    password=db_password
)



In [ ]:
dataset_str = "English"
query = f"""
SELECT sample.* 
FROM sample
JOIN dataset ON sample.dataset_id = dataset.id
WHERE dataset.name='{dataset_str}' 
    AND sample.trimmed_audio_duration > 0 
    AND sample.trimmed_audio_duration < 5 
    AND sample.longest_pause < 0.5
    AND sample.is_selected_for_delivery = TRUE
"""



# Assume you've loaded the sample and annotation tables as DataFrames
sample_df = pd.read_sql(query, connection)
annotation_df = pd.read_sql("SELECT * FROM annotation", connection)

# Merge the DataFrames based on the sample.id and annotation.sample_id
merged_df = pd.merge(sample_df, annotation_df, left_on='id', right_on='sample_id', how='left')


In [ ]:
merged_df = merged_df[~merged_df.status.isnull()]

# drop duplicates in id_x and select first row
merged_df = merged_df.drop_duplicates(subset=['id_x'], keep='first')



In [ ]:
# merged_df = merged_df[["filename", "original_text", "final_text"]]

In [ ]:
merged_df["is_edited"] = False
# merged_df[merged_df.original_text != merged_df.final_text]["is_edited"] = True
merged_df.loc[merged_df.original_text != merged_df.final_text, "is_edited"] = True
# if status = Discarded then is_edited = False
merged_df.loc[merged_df.status == "Discarded", "is_edited"] = False


In [ ]:
merged_df.shape

In [ ]:
merged_df[merged_df.is_edited == True][["filename", "original_text", "final_text"]].head(10)

In [ ]:
merged_df["is_edited"].value_counts(normalize=True)

In [ ]:
merged_df.status.value_counts(normalize=True)

In [ ]:
# calculate the duration of the not discarded samples
merged_df[merged_df.status != "Discarded"].trimmed_audio_duration.sum() / 60 / 60